In [1]:
import numpy as np
import pandas as pd

In [2]:
import content_rocchio

In [3]:
ratings = pd.read_json('./ratings.jsonl', lines=True)
content = pd.read_json('./content.jsonl', lines=True)
targets = pd.read_csv('targets.csv', encoding='latin-1', sep=',')

In [4]:
_content = content[['ItemId', 'Actors', 'Director', 'Genre', 'Title', 'imdbVotes']]
_content['Actors'] = _content.Actors.astype(str)
# df_1['plot_keywords'] = df_1.plot_keywords.astype(str) # get keywords from the 'Plot' field
_content['Genre'] = _content.Genre.astype(str)
_content['Director'] = _content.Director.astype(str)


# Removing spaces between names
_content['Director'] = _content['Director'].apply(lambda x: ' '.join(x.replace(' ', '').lower() for x in x.split(', ')))
_content['Actors'] = _content['Actors'].apply(lambda x: ' '.join(x.replace(' ', '').lower() for x in x.split(', ')))

# Discarding the pipes between the plot keywords' and getting only the first five words
# df_1['plot_keywords'] = df_1['plot_keywords'].map(lambda x: x.split('|')) # 'Plot' field but only the keywords
# df_1['plot_keywords'] = df_1['plot_keywords'].apply(lambda x: " ".join(x)) #  'Plot' field but only the keywords

# Discarding the pipes between the genres 
_content['Genre'] = _content['Genre'].map(lambda x: x.lower().replace(', ', ' '))
# _content['Features'] = _content['Actors'] +' '+ _content['Director'] +' '+ _content['Genre']
_content['Features'] = _content['Genre']

_content['imdbVotes'] = _content['imdbVotes'].str.replace('N/A', '1', regex=True)
_content['imdbVotes'] = _content['imdbVotes'].str.replace(',', '', regex=True)
_content['imdbVotes'] = _content['imdbVotes'].astype(str).astype('float64')

/tmp/ipykernel_36093/1151917588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _content['Actors'] = _content.Actors.astype(str)
/tmp/ipykernel_36093/1151917588.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _content['Genre'] = _content.Genre.astype(str)
/tmp/ipykernel_36093/1151917588.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [5]:
rocchio_recommender = content_rocchio.Rocchio(ratings)
rocchio_recommender.fit(_content[['ItemId', 'Features']])


AttributeError: 'Rocchio' object has no attribute 'user_utility'

In [ ]:
recommendations_ = rocchio_recommender.predict(targets)
recommendations_ = recommendations_.merge(_content[['ItemId', 'imdbVotes']], on='ItemId')
recommendations_['Similarity'] = recommendations_['Similarity']*recommendations_['imdbVotes']
recommendations_ = recommendations_.sort_values(['UserId','Similarity','imdbVotes'], ascending=[True, False, False])
output = recommendations_[['UserId', 'ItemId']].copy()

output_file = open('out.csv','w')
print('UserId,ItemId', file=output_file)
pd.set_option("display.max_rows", None, "display.max_columns", None)
output = output.to_numpy()
for user, item in output:
    print('{},{}'.format(user,item), file=output_file)

KeyError: 'UserId'